In [72]:
#há uma descrição da conversao de texto em vetores no site do scikit learn. 

#o vetor não terá os IDs, só o pandas

import pandas

texto = pandas.read_csv('training.txt', sep='\t', header=None)
texto.shape

(6918, 2)

In [73]:
texto.sample(5)

,0,1
5422,0,I hate Harry Potter.
5276,0,"I hate Harry Potter, that daniel wotshisface n..."
5969,0,Brokeback mountain sucks balls....
2439,1,I love Harry Potter.
5940,0,"Always knows what I want, not guy crazy, hates..."


In [91]:
# especificação das classes
target = texto[0]
print(target.shape)
data = texto.drop(0, 1)
print(data.shape)
#data[1].values
data[1].values

(6918,)
(6918, 1)


array(['The Da Vinci Code book is just awesome.',
       "this was the first clive cussler i've ever read, but even books like Relic, and Da Vinci code were more plausible than this.",
       'i liked the Da Vinci Code a lot.', ...,
       'As I sit here, watching the MTV Movie Awards, I am reminded of how much I despised the movie Brokeback Mountain.',
       'Ok brokeback mountain is such a horrible movie.',
       'Oh, and Brokeback Mountain was a terrible movie.'], dtype=object)

In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data[1].values)
vectors.shape

(6918, 2132)

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

clf = MultinomialNB(alpha = .01)
clf.fit(vectors, classe)
pred = clf.predict(vectors[7:])
print(metrics.classification_report(classe[7:], pred))

IndexError: index out of bounds: 0 <= 7 <= 2, 0 <= 2 <= 2, 7 <= 2